# Preprocessing customers.csv Part 4

In [1]:
import numpy as np
import pandas as pd
import boto3
import matplotlib.pyplot as plt
import seaborn as sns
import os
from numpy import inf

pd.options.display.float_format = '{:.4f}'.format

### Loading Data via S3

In [ ]:
bucket='ads-508-group-6-raw'
data_key = 'customers_processed_part_1'

s3 = boto3.client('s3')
customers_obj = s3.get_object(Bucket = bucket, Key = data_key)

customers_df = pd.read_csv(customers_obj['Body'])

### Loading Data Locally

In [2]:
customers_df = pd.read_csv("C:\\Users\\garyb\\OneDrive\\Documents\\GitHub\\recommendation-engine\\data\\customers_processed_part_3.csv")

In [3]:
customers_df.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code,churn,lifetime_days_min_max,lifetime_articles_purchased,lifetime_amount_spent,lifetime_sales_channel_1_proportion,lifetime_average_amount_spent_per_article,lifetime_average_days_between_purchase,quarter_days_min_max,quarter_articles_purchased,quarter_amount_spent,quarter_sales_channel_1_proportion,quarter_average_amount_spent_per_article,quarter_average_days_between_purchase
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0.0000,0.0000,ACTIVE,NONE,49.0000,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...,1.0000,450,20,0.5982,1.0000,0.0299,0.0013,0,5,0.0936,1.0000,0.0187,0.0000
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0.0000,0.0000,ACTIVE,NONE,25.0000,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...,0.0000,527,71,2.2378,1.0000,0.0315,0.0042,40,19,0.4726,1.0000,0.0249,0.0118
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0.0000,0.0000,ACTIVE,NONE,24.0000,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...,0.0000,501,7,0.2151,1.0000,0.0307,0.0004,0,3,0.0694,1.0000,0.0231,0.0000
3,00007e8d4e54114b5b2a9b51586325a8d0fa74ea23ef77...,0.0000,0.0000,ACTIVE,NONE,20.0000,2c29ae653a9282cce4151bd87643c907644e09541abc28...,1.0000,0,2,0.0534,1.0000,0.0267,0.0000,0,2,0.0534,1.0000,0.0267,0.0000
4,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,1.0000,1.0000,ACTIVE,Regularly,56.0000,b31984b20a8c478de38eaf113c581ff64e63c4242e607b...,0.0000,526,111,3.5527,1.0000,0.0320,0.0068,71,21,0.6582,1.0000,0.0313,0.0093


### Data Transformation

In [4]:
cat_variables = ['club_member_status','fashion_news_frequency']
num_variables = ['age', 
                 'lifetime_days_min_max', 
                 'lifetime_articles_purchased', 
                 'lifetime_amount_spent', 
                 'lifetime_average_amount_spent_per_article', 
                 'lifetime_average_days_between_purchase', 
                 'quarter_days_min_max', 
                 'quarter_articles_purchased', 
                 'quarter_amount_spent', 
                 'quarter_average_amount_spent_per_article', 
                 'quarter_average_days_between_purchase']

In [5]:
cat_dummies = pd.get_dummies(customers_df[cat_variables])

In [6]:
def mm_normalize(df,col_list):
    temp_df = pd.DataFrame()
    for col in col_list:
        temp_df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())
    return temp_df

In [7]:
num_normed = mm_normalize(customers_df, num_variables)

In [10]:
processed_df = pd.concat([customers_df['churn'], 
                          customers_df['FN'], 
                          customers_df['Active'],
                          cat_dummies, 
                          num_normed,
                          customers_df['lifetime_sales_channel_1_proportion'],
                          customers_df['quarter_sales_channel_1_proportion']],
                         axis = 1)

In [11]:
customers_df.to_csv("C:\\Users\\garyb\\OneDrive\\Documents\\GitHub\\recommendation-engine\\data\\customers_processed_part_4.csv", 
                    index = False)